### Successful cleaning

In [ ]:
import ast
import pandas as pd

# Prep data for explosion/normalization
df = pd.read_csv('../data/raw/NBAodds2025-11-10.csv')

def safe_parse(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return None
    return x

df['bookmakers'] = df['bookmakers'].apply(safe_parse)

In [ ]:
# 1) explode bookmakers
df1 = df.explode('bookmakers').reset_index(drop=True)

# flatten bookmaker dict
bk = pd.json_normalize(df1['bookmakers'])
bk = bk.rename(columns={
    'key': 'bookmaker_key',
    'title': 'bookmaker_title',
    'last_update': 'bookmaker_last_update'
})
df1 = pd.concat([df1.drop(columns='bookmakers'), bk], axis=1)


# 2) explode markets
df2 = df1.explode('markets').reset_index(drop=True)

# flatten market dict
mkt = pd.json_normalize(df2['markets'])
mkt = mkt.rename(columns={
    'key': 'market_key',
    'last_update': 'market_last_update'
})
df2 = pd.concat([df2.drop(columns='markets'), mkt], axis=1)


# 3) explode outcomes
df3 = df2.explode('outcomes').reset_index(drop=True)

# flatten outcome dict
out = pd.json_normalize(df3['outcomes'])
# typically no conflicting names here

final = pd.concat([df3.drop(columns='outcomes'), out], axis=1)

In [4]:
final.head()

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,name,price,point
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Detroit Pistons,1.89,-11.5
1,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Washington Wizards,1.93,11.5
2,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Detroit Pistons,1.90,-11.5
3,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Washington Wizards,1.99,11.5
4,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Detroit Pistons,1.87,-11.5


In [5]:
len(df), len(df1), len(df2), len(df3), len(final)


(12, 143, 143, 286, 286)

In [6]:
df

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
1,3f7e2359b4a5532aafa274ff22714d51,basketball_nba,NBA,2025-11-11T00:10:00Z,Charlotte Hornets,Los Angeles Lakers,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
2,6fb85302d766cc4bfcd431455d308748,basketball_nba,NBA,2025-11-11T00:10:00Z,Orlando Magic,Portland Trail Blazers,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
3,bc264cf0d07f01c785e02a8b9e27f59b,basketball_nba,NBA,2025-11-11T00:30:00Z,Miami Heat,Cleveland Cavaliers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
4,404b446a92bde56ee3697b9a2695e468,basketball_nba,NBA,2025-11-11T01:10:00Z,Chicago Bulls,San Antonio Spurs,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
5,4f07ee458d197bc2ccdcfcd9934ffc03,basketball_nba,NBA,2025-11-11T01:30:00Z,Dallas Mavericks,Milwaukee Bucks,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
6,eea6f48cf04bc26e631209e50a4a3af1,basketball_nba,NBA,2025-11-11T02:00:00Z,Utah Jazz,Minnesota Timberwolves,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
7,ed82190bf1c73a6310c46ee399696436,basketball_nba,NBA,2025-11-11T02:10:00Z,Phoenix Suns,New Orleans Pelicans,"[{'key': 'fanduel', 'title': 'FanDuel', 'last_..."
8,1712f1f48109a3c9ef509a77914aaa06,basketball_nba,NBA,2025-11-11T03:30:00Z,Los Angeles Clippers,Atlanta Hawks,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
9,f557c9173a46a3c7c87afb4bd68f6644,basketball_nba,NBA,2025-12-25T17:00:00Z,New York Knicks,Cleveland Cavaliers,"[{'key': 'ballybet', 'title': 'Bally Bet', 'la..."


In [7]:
df1

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,markets
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
1,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
2,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
3,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,mybookieag,MyBookie.ag,2025-11-10T19:29:58Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
4,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,fliff,Fliff,2025-11-10T19:29:56Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
...,...,...,...,...,...,...,...,...,...,...
138,9010f7d8f26d5880c63b67355800f997,basketball_nba,NBA,2025-12-25T22:00:00Z,Golden State Warriors,Dallas Mavericks,betparx,betPARX,2025-11-10T19:30:20Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
139,9010f7d8f26d5880c63b67355800f997,basketball_nba,NBA,2025-12-25T22:00:00Z,Golden State Warriors,Dallas Mavericks,betrivers,BetRivers,2025-11-10T19:29:44Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
140,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,ballybet,Bally Bet,2025-11-10T19:30:07Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."
141,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,"[{'key': 'spreads', 'last_update': '2025-11-10..."


In [8]:
df2

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcomes
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,"[{'name': 'Detroit Pistons', 'price': 1.89, 'p..."
1,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,"[{'name': 'Detroit Pistons', 'price': 1.9, 'po..."
2,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"[{'name': 'Detroit Pistons', 'price': 1.87, 'p..."
3,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,mybookieag,MyBookie.ag,2025-11-10T19:29:58Z,spreads,2025-11-10T19:29:58Z,"[{'name': 'Detroit Pistons', 'price': 1.91, 'p..."
4,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,fliff,Fliff,2025-11-10T19:29:56Z,spreads,2025-11-10T19:29:56Z,"[{'name': 'Detroit Pistons', 'price': 1.91, 'p..."
...,...,...,...,...,...,...,...,...,...,...,...,...
138,9010f7d8f26d5880c63b67355800f997,basketball_nba,NBA,2025-12-25T22:00:00Z,Golden State Warriors,Dallas Mavericks,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"[{'name': 'Dallas Mavericks', 'price': 1.92, '..."
139,9010f7d8f26d5880c63b67355800f997,basketball_nba,NBA,2025-12-25T22:00:00Z,Golden State Warriors,Dallas Mavericks,betrivers,BetRivers,2025-11-10T19:29:44Z,spreads,2025-11-10T19:29:44Z,"[{'name': 'Dallas Mavericks', 'price': 1.89, '..."
140,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,ballybet,Bally Bet,2025-11-10T19:30:07Z,spreads,2025-11-10T19:30:07Z,"[{'name': 'Denver Nuggets', 'price': 1.88, 'po..."
141,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"[{'name': 'Denver Nuggets', 'price': 1.88, 'po..."


In [9]:
df3

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,outcomes
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,"{'name': 'Detroit Pistons', 'price': 1.89, 'po..."
1,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,"{'name': 'Washington Wizards', 'price': 1.93, ..."
2,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,"{'name': 'Detroit Pistons', 'price': 1.9, 'poi..."
3,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,"{'name': 'Washington Wizards', 'price': 1.99, ..."
4,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"{'name': 'Detroit Pistons', 'price': 1.87, 'po..."
...,...,...,...,...,...,...,...,...,...,...,...,...
281,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,ballybet,Bally Bet,2025-11-10T19:30:07Z,spreads,2025-11-10T19:30:07Z,"{'name': 'Minnesota Timberwolves', 'price': 1...."
282,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"{'name': 'Denver Nuggets', 'price': 1.88, 'poi..."
283,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,"{'name': 'Minnesota Timberwolves', 'price': 1...."
284,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betrivers,BetRivers,2025-11-10T19:29:44Z,spreads,2025-11-10T19:29:44Z,"{'name': 'Denver Nuggets', 'price': 1.85, 'poi..."


In [10]:
final

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,name,price,point
0,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Detroit Pistons,1.89,-11.5
1,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Washington Wizards,1.93,11.5
2,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Detroit Pistons,1.90,-11.5
3,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Washington Wizards,1.99,11.5
4,f2e9b932ad12be3f2f47166b3eb0d456,basketball_nba,NBA,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Detroit Pistons,1.87,-11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,ballybet,Bally Bet,2025-11-10T19:30:07Z,spreads,2025-11-10T19:30:07Z,Minnesota Timberwolves,1.93,5.0
282,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Denver Nuggets,1.88,-5.0
283,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Minnesota Timberwolves,1.93,5.0
284,6b9d46fb1b22af30b771b5f6b8a4c6f8,basketball_nba,NBA,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betrivers,BetRivers,2025-11-10T19:29:44Z,spreads,2025-11-10T19:29:44Z,Denver Nuggets,1.85,-5.0


In [11]:
final = final.drop(columns=['sport_key', 'sport_title'])
final 

,id,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,market_last_update,name,price,point
0,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Detroit Pistons,1.89,-11.5
1,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,2025-11-10T19:28:57Z,Washington Wizards,1.93,11.5
2,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Detroit Pistons,1.90,-11.5
3,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,2025-11-10T19:30:31Z,Washington Wizards,1.99,11.5
4,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Detroit Pistons,1.87,-11.5
...,...,...,...,...,...,...,...,...,...,...,...,...
281,6b9d46fb1b22af30b771b5f6b8a4c6f8,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,ballybet,Bally Bet,2025-11-10T19:30:07Z,spreads,2025-11-10T19:30:07Z,Minnesota Timberwolves,1.93,5.0
282,6b9d46fb1b22af30b771b5f6b8a4c6f8,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Denver Nuggets,1.88,-5.0
283,6b9d46fb1b22af30b771b5f6b8a4c6f8,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betparx,betPARX,2025-11-10T19:30:20Z,spreads,2025-11-10T19:30:20Z,Minnesota Timberwolves,1.93,5.0
284,6b9d46fb1b22af30b771b5f6b8a4c6f8,2025-12-26T03:30:00Z,Denver Nuggets,Minnesota Timberwolves,betrivers,BetRivers,2025-11-10T19:29:44Z,spreads,2025-11-10T19:29:44Z,Denver Nuggets,1.85,-5.0


In [12]:
final.isna().sum(), final.nunique()

(id                       0
 commence_time            0
 home_team                0
 away_team                0
 bookmaker_key            0
 bookmaker_title          0
 bookmaker_last_update    0
 market_key               0
 market_last_update       0
 name                     0
 price                    0
 point                    0
 dtype: int64,
 id                       12
 commence_time            11
 home_team                12
 away_team                10
 bookmaker_key            15
 bookmaker_title          15
 bookmaker_last_update    11
 market_key                1
 market_last_update       11
 name                     21
 price                    15
 point                    30
 dtype: int64)

In [13]:
#market key is useless, always "spreads". duh
final = final.drop(columns=['market_last_update'])

lets rename the current id to be more useful (or maybe drop it), and more importantly create a unique combination key,

In [14]:
unique_key = final['id'] + '_' + final['bookmaker_key']  + '_' + final['name']
final['unique_key'] = unique_key
final.head()

,id,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,name,price,point,unique_key
0,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,Detroit Pistons,1.89,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_draftkings_De...
1,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,Washington Wizards,1.93,11.5,f2e9b932ad12be3f2f47166b3eb0d456_draftkings_Wa...
2,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,Detroit Pistons,1.90,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_lowvig_Detroi...
3,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,Washington Wizards,1.99,11.5,f2e9b932ad12be3f2f47166b3eb0d456_lowvig_Washin...
4,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,Detroit Pistons,1.87,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_betonlineag_D...


In [15]:
unique_key[0]

'f2e9b932ad12be3f2f47166b3eb0d456_draftkings_Detroit Pistons'

In [16]:
#rename id to gameid
final = final.rename(columns={'id': 'game_id'})
final.head()

,game_id,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,bookmaker_last_update,market_key,name,price,point,unique_key
0,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,Detroit Pistons,1.89,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_draftkings_De...
1,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,draftkings,DraftKings,2025-11-10T19:28:57Z,spreads,Washington Wizards,1.93,11.5,f2e9b932ad12be3f2f47166b3eb0d456_draftkings_Wa...
2,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,Detroit Pistons,1.90,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_lowvig_Detroi...
3,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,lowvig,LowVig.ag,2025-11-10T19:30:31Z,spreads,Washington Wizards,1.99,11.5,f2e9b932ad12be3f2f47166b3eb0d456_lowvig_Washin...
4,f2e9b932ad12be3f2f47166b3eb0d456,2025-11-11T00:00:00Z,Detroit Pistons,Washington Wizards,betonlineag,BetOnline.ag,2025-11-10T19:30:20Z,spreads,Detroit Pistons,1.87,-11.5,f2e9b932ad12be3f2f47166b3eb0d456_betonlineag_D...
